In [23]:
# Load all the dependencies
import os
import sys
import random
import warnings
import numpy as np
from itertools import chain
from numpy import genfromtxt
from tensorflow import random
from keras import backend as K
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.layers import Layer, UpSampling2D, GlobalAveragePooling2D, Multiply, Dense, Reshape, Permute, multiply, dot, add, Input
from keras.layers.core import Dropout, Lambda, SpatialDropout2D, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model, load_model, model_from_yaml, Sequential
import tensorflow as tf

np.random.seed(1337) # for reproducibility
random.set_seed(1337)
print(tf.__version__)

name = "model_train_from_scratch_"

2.1.0


In [24]:
# Use dice coefficient function as the loss function 
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2.0 * intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1.0)

# Jacard coefficient
def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)

# calculate loss value
def jacard_coef_loss(y_true, y_pred):
    return -jacard_coef(y_true, y_pred)

# calculate loss value
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

def Residual_CNN_block(x, size, dropout=0.0, batch_norm=True):
    if K.image_data_format() == 'th':
        axis = 1
    else:
        axis = 3
    conv = Conv2D(size, (3, 3), padding='same')(x)
    if batch_norm is True:
        conv = BatchNormalization(axis=axis)(conv)
    conv = Activation('relu')(conv)
    conv = Conv2D(size, (3, 3), padding='same')(conv)
    if batch_norm is True:
        conv = BatchNormalization(axis=axis)(conv)
    conv = Activation('relu')(conv)
    conv = Conv2D(size, (3, 3), padding='same')(conv)
    if batch_norm is True:
        conv = BatchNormalization(axis=axis)(conv)
    conv = Activation('relu')(conv)
    return conv

class multiplication(Layer):
    def __init__(self,inter_channel = None,**kwargs):
        super(multiplication, self).__init__(**kwargs)
        self.inter_channel = inter_channel
    def build(self,input_shape=None):
        self.k = self.add_weight(name='k',shape=(1,),initializer='zeros',dtype='float32',trainable=True)
    def get_config(self):
        base_config = super(multiplication, self).get_config()
        config = {'inter_channel':self.inter_channel}
        return dict(list(base_config.items()) + list(config.items()))  
    def call(self,inputs):
        g,x,x_query,phi_g,x_value = inputs[0],inputs[1],inputs[2],inputs[3],inputs[4]
        h,w,c = int(x.shape[1]),int(x.shape[2]),int(x.shape[3])
        x_query = K.reshape(x_query, shape=(-1,h*w, self.inter_channel//4))
        phi_g = K.reshape(phi_g,shape=(-1,h*w,self.inter_channel//4))
        x_value = K.reshape(x_value,shape=(-1,h*w,c))
        scale = dot([K.permute_dimensions(phi_g,(0,2,1)), x_query], axes=(1, 2))
        soft_scale = Activation('softmax')(scale)
        scaled_value = dot([K.permute_dimensions(soft_scale,(0,2,1)),K.permute_dimensions(x_value,(0,2,1))],axes=(1, 2))
        scaled_value = K.reshape(scaled_value, shape=(-1,h,w,c))        
        customize_multi = self.k * scaled_value
        layero = add([customize_multi,x])
        my_concat = Lambda(lambda x: K.concatenate([x[0], x[1]], axis=3))
        concate = my_concat([layero,g])
        return concate 
    def compute_output_shape(self,input_shape):
        ll = list(input_shape)[1]
        return (None,ll[1],ll[1],ll[3]*3)
    def get_custom_objects():
        return {'multiplication': multiplication}

def attention_up_and_concatenate(inputs):
    g,x = inputs[0],inputs[1]
    inter_channel = g.get_shape().as_list()[3]
    g = Conv2DTranspose(inter_channel, (2,2), strides=[2, 2],padding='same')(g)
    x_query = Conv2D(inter_channel//4, [1, 1], strides=[1, 1], data_format='channels_last')(x)
    phi_g = Conv2D(inter_channel//4, [1, 1], strides=[1, 1], data_format='channels_last')(g)
    x_value = Conv2D(inter_channel//2, [1, 1], strides=[1, 1], data_format='channels_last')(x)
    inputs = [g,x,x_query,phi_g,x_value]
    concate = multiplication(inter_channel)(inputs)
    return concate

class multiplication2(Layer):
    def __init__(self,inter_channel = None,**kwargs):
        super(multiplication2, self).__init__(**kwargs)
        self.inter_channel = inter_channel
    def build(self,input_shape=None):
        self.k = self.add_weight(name='k',shape=(1,),initializer='zeros',dtype='float32',trainable=True)
    def get_config(self):
        base_config = super(multiplication2, self).get_config()
        config = {'inter_channel':self.inter_channel}
        return dict(list(base_config.items()) + list(config.items()))  
    def call(self,inputs):
        g,x,rate = inputs[0],inputs[1],inputs[2]
        scaled_value = multiply([x, rate])
        att_x =  self.k * scaled_value
        att_x = add([att_x,x])
        my_concat = Lambda(lambda x: K.concatenate([x[0], x[1]], axis=3))
        concate = my_concat([att_x, g])
        return concate 
    def compute_output_shape(self,input_shape):
        ll = list(input_shape)[1]
        return (None,ll[1],ll[1],ll[3]*2)
    def get_custom_objects():
        return {'multiplication2': multiplication2}

def attention_up_and_concatenate2(inputs):
    g, x = inputs[0],inputs[1]
    inter_channel = g.get_shape().as_list()[3]
    g = Conv2DTranspose(inter_channel//2, (3,3), strides=[2, 2],padding='same')(g)
    g = Conv2D(inter_channel//2, [1, 1], strides=[1, 1], data_format='channels_last')(g)
    theta_x = Conv2D(inter_channel//4, [1, 1], strides=[1, 1], data_format='channels_last')(x)
    phi_g = Conv2D(inter_channel//4, [1, 1], strides=[1, 1], data_format='channels_last')(g)
    f = Activation('relu')(add([theta_x, phi_g]))
    psi_f = Conv2D(1, [1, 1], strides=[1, 1], data_format='channels_last')(f)
    rate = Activation('sigmoid')(psi_f)
    concate =  multiplication2()([g,x,rate])
    return concate

In [25]:
# The structure of the constructed U-net model
def UNET_224(weights=None):
    if K.image_data_format() == 'th':
        inputs = Input((INPUT_CHANNELS, IMG_WIDTH, IMG_WIDTH))
        axis = 1
    else:
        inputs = Input((IMG_WIDTH, IMG_WIDTH, INPUT_CHANNELS))
        axis = 3
    filters = 32
    last_dropout = 0.2
# convolutiona and pooling level 1
    conv_224 = Residual_CNN_block(inputs,filters)
    pool_112 = MaxPooling2D(pool_size=(2, 2))(conv_224)
# convolutiona and pooling level 2
    conv_112 = Residual_CNN_block(pool_112,2*filters)
    pool_56 = MaxPooling2D(pool_size=(2, 2))(conv_112)
# convolutiona and pooling level 3
    conv_56 = Residual_CNN_block(pool_56,4*filters)
    pool_28 = MaxPooling2D(pool_size=(2, 2))(conv_56)
# convolutiona and pooling level 4
    conv_28 = Residual_CNN_block(pool_28,8*filters)
    pool_14 = MaxPooling2D(pool_size=(2, 2))(conv_28)
# convolutiona and pooling level 5
    conv_14 = Residual_CNN_block(pool_14,16*filters)
    pool_7 = MaxPooling2D(pool_size=(2, 2))(conv_14)
# Conlovlution and feature concatenation
    conv_7 = Residual_CNN_block(pool_7,32*filters)
# Upsampling with convolution 
    up_14 = attention_up_and_concatenate([conv_7, conv_14]) 
    up_conv_14 = Residual_CNN_block(up_14,16*filters)
# Upsampling with convolution 2
    up_28 = attention_up_and_concatenate([up_conv_14, conv_28])
    up_conv_28 = Residual_CNN_block(up_28,8*filters)
# Upsampling with convolution 3
    up_56 = attention_up_and_concatenate2([up_conv_28, conv_56])
    up_conv_56 = Residual_CNN_block(up_56,4*filters)
# Upsampling with convolution 4
    up_112 = attention_up_and_concatenate2([up_conv_56, conv_112])
    up_conv_112 = Residual_CNN_block(up_112,2*filters)
# Upsampling with convolution 5
    up_224 = attention_up_and_concatenate2([up_conv_112, conv_224])
    #up_224 = attention_up_and_concatenate2(up_conv_112, conv_224)
    up_conv_224 = Residual_CNN_block(up_224,filters,dropout = last_dropout)
# 1 dimensional convolution and generate probabilities from Sigmoid function
    conv_final = Conv2D(OUTPUT_MASK_CHANNELS, (1, 1))(up_conv_224)  
    conv_final = Activation('sigmoid')(conv_final)
# Generate model
    model = Model(inputs, conv_final, name="UNET_224")
    return model

## **Reading sample**

In [26]:
name = "model_train_from_scratch_"
name += 'No_NAIP_'

# The dataset has 9 channels:
# 0. Curvature
# 1. Slope
# 2. Openness
# 3. DEM
# 4. TPI 21
# 5. Reflectance (LiDAR intensity)
# 6. Geomorphon
# 7. TPI 9
# 8. TPI 3

# round_num = 10
# sample_size = 100

# data_path = './samples/'+str(sample_size)+'/'+str(round_num)+'/'
# # read in training and validation data
# X_train_new = np.load(data_path+'train_data.npy')
# print(X_train_new.shape)
# Y_train = np.load(data_path+'train_label.npy')
# print(Y_train.shape)
# X_Validation_new = np.load(data_path+'vali_data.npy')
# print(X_Validation_new.shape)
# Y_Validation = np.load(data_path+'vali_label.npy')
# print(Y_Validation.shape)
# name += str(sample_size)+"_"
# name += "samples_r"+str(round_num)+"_"

# The 300 training + 300 validation samples files
# data_path = './samples/'
# read in training and validation data
# X_train = np.load(data_path+'train_data.npy')
# Y_train = np.load(data_path+'train_label.npy')
# X_Validation = np.load(data_path+'vali_data.npy')
# Y_Validation = np.load(data_path+'vali_label.npy')

# augnebted data
data_path = '../Covington_data/without_NAIP/nodata_as_0/augmented_top_half/'
X_train_new = np.load(data_path+'train_data_aug.npy')
print(X_train_new.shape)
Y_train = np.load(data_path+'train_label_aug.npy')
print(Y_train.shape)
X_Validation_new = np.load(data_path+'vali_data.npy')
print(X_Validation_new.shape)
Y_Validation = np.load(data_path+'vali_label.npy')
print(Y_Validation.shape)
name += "top_half_augmented_"

(1750, 224, 224, 8)
(1750, 224, 224, 1)
(160, 224, 224, 8)
(160, 224, 224, 1)


In [28]:
patch_size = 224
IMG_WIDTH = patch_size
IMG_HEIGHT = patch_size
# Number of feature channels 
INPUT_CHANNELS = 8
# Number of output masks (1 in case you predict only one type of objects)
OUTPUT_MASK_CHANNELS = 1
maxepoch = 100
# hyperparameters
learning_rate = 0.0000359
# learning_rate = 0.0001
patience = 20
model = UNET_224()
model.compile(optimizer=Adam(lr=learning_rate),loss = dice_coef_loss,metrics=[dice_coef,'accuracy'])
callbacks = [
        ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=patience, min_lr=1e-9, verbose=1, mode='min'),
        EarlyStopping(monitor='val_loss', patience=patience+10, verbose=0),
        ModelCheckpoint(name+'_save.h5', monitor='val_loss', save_best_only=True, verbose=0),
    ]

# name += str(maxepoch)+'_epoch_'
print(name)

model_train_from_scratch_No_NAIP_top_half_augmented_


In [29]:
no_transfer_learning_results = model.fit(X_train_new, Y_train, validation_data=(X_Validation_new,Y_Validation), batch_size=2, epochs=maxepoch, callbacks=callbacks)

Train on 1750 samples, validate on 160 samples
Epoch 1/100
1750/1750 [==============================] - 131s 75ms/step - loss: -0.2676 - dice_coef: 0.2676 - accuracy: 0.8813 - val_loss: -0.1946 - val_dice_coef: 0.1946 - val_accuracy: 0.9807
Epoch 2/100
1750/1750 [==============================] - 121s 69ms/step - loss: -0.3821 - dice_coef: 0.3821 - accuracy: 0.9481 - val_loss: -0.2576 - val_dice_coef: 0.2576 - val_accuracy: 0.9780
Epoch 3/100
1750/1750 [==============================] - 121s 69ms/step - loss: -0.4636 - dice_coef: 0.4636 - accuracy: 0.9582 - val_loss: -0.3063 - val_dice_coef: 0.3063 - val_accuracy: 0.9741
Epoch 4/100
1750/1750 [==============================] - 121s 69ms/step - loss: -0.5354 - dice_coef: 0.5354 - accuracy: 0.9635 - val_loss: -0.3781 - val_dice_coef: 0.3781 - val_accuracy: 0.9823
Epoch 5/100
1750/1750 [==============================] - 121s 69ms/step - loss: -0.5985 - dice_coef: 0.5985 - accuracy: 0.9673 - val_loss: -0.4261 - val_dice_coef: 0.4261 - val_

In [30]:
import pickle
import time
timestr = time.strftime("%Y%m%d-%H%M%S")
# name += timestr
# root_path = './training_results/'+str(sample_size)+'/'

root_path = './training_results/'

# save the trained model
if not os.path.exists(root_path):
        os.makedirs(root_path)
        
model_yaml = model.to_yaml()
with open(root_path+name+".yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
# save the weights
model.save(root_path+name+".h5")
# save the intermdediate results and training statistics
with open(root_path+name+".pickle", 'wb') as file_pi:
    pickle.dump(no_transfer_learning_results.history, file_pi, protocol=2)

from IPython.display import Audio
sound_file = '../../wink.mp3'

Audio(sound_file, autoplay=True)

In [31]:
# name += 'No_NAIP_'

# data_path = './samples/'
# # read in training and validation data
# X_train = np.load(data_path+'train_data.npy')
# Y_train = np.load(data_path+'train_label.npy')
# X_Validation = np.load(data_path+'vali_data.npy')
# Y_Validation = np.load(data_path+'vali_label.npy')

# The dataset has 9 channels:
# 0. Curvature
# 1. Slope
# 2. Openness
# 3. DEM
# 4. TPI 21
# 5. Reflectance (LiDAR intensity)
# 6. Geomorphon
# 7. TPI 9
# 8. TPI 3

# # ##############################################
# # 100 samples 
# X_train_new = X_train[0:80,:,:,:]
# print(X_train_new.shape)
# Y_train = np.load(data_path+'train_label.npy')[0:80,:,:,:]
# print(Y_train.shape)

# X_Validation_new = X_Validation[0:20,:,:,:]
# print(X_Validation_new.shape)
# Y_Validation = np.load(data_path+'vali_label.npy')[0:20,:,:,:]
# print(Y_Validation.shape)
# name += "100_samples"
# # ##############################################

# # ##############################################
# # 90 samples 
# X_train_new = X_train[0:72,:,:,:]
# print(X_train_new.shape)
# Y_train = np.load(data_path+'train_label.npy')[0:72,:,:,:]
# print(Y_train.shape)

# X_Validation_new = X_Validation[0:18,:,:,:]
# print(X_Validation_new.shape)
# Y_Validation = np.load(data_path+'vali_label.npy')[0:18,:,:,:]
# print(Y_Validation.shape)
# name += "90_samples_"
# # ##############################################

# # ##############################################
# # 80 samples 
# X_train_new = X_train[0:64,:,:,:]
# print(X_train_new.shape)
# Y_train = np.load(data_path+'train_label.npy')[0:64,:,:,:]
# print(Y_train.shape)

# X_Validation_new = X_Validation[0:16,:,:,:]
# print(X_Validation_new.shape)
# Y_Validation = np.load(data_path+'vali_label.npy')[0:16,:,:,:]
# print(Y_Validation.shape)
# name += "80_samples_"
# # ##############################################

# # ##############################################
# # 70 samples 
# X_train_new = X_train[0:56,:,:,:]
# print(X_train_new.shape)
# Y_train = np.load(data_path+'train_label.npy')[0:56,:,:,:]
# print(Y_train.shape)

# X_Validation_new = X_Validation[0:14,:,:,:]
# print(X_Validation_new.shape)
# Y_Validation = np.load(data_path+'vali_label.npy')[0:14,:,:,:]
# print(Y_Validation.shape)
# name += "70_samples_"
# # ##############################################

# # ##############################################
# # 60 samples 
# X_train_new = X_train[0:48,:,:,:]
# print(X_train_new.shape)
# Y_train = np.load(data_path+'train_label.npy')[0:48,:,:,:]
# print(Y_train.shape)

# X_Validation_new = X_Validation[0:12,:,:,:]
# print(X_Validation_new.shape)
# Y_Validation = np.load(data_path+'vali_label.npy')[0:12,:,:,:]
# print(Y_Validation.shape)
# name += "60_samples_"
# # ##############################################

# # ##############################################
# # 50 samples 
# X_train_new = X_train[0:40,:,:,:]
# print(X_train_new.shape)
# Y_train = np.load(data_path+'train_label.npy')[0:40,:,:,:]
# print(Y_train.shape)

# X_Validation_new = X_Validation[0:10,:,:,:]
# print(X_Validation_new.shape)
# Y_Validation = np.load(data_path+'vali_label.npy')[0:10,:,:,:]
# print(Y_Validation.shape)
# name += "50_samples_"
# # ##############################################

# # ##############################################
# # 40 samples 
# X_train_new = X_train[0:32,:,:,:]
# print(X_train_new.shape)
# Y_train = np.load(data_path+'train_label.npy')[0:32,:,:,:]
# print(Y_train.shape)

# X_Validation_new = X_Validation[0:8,:,:,:]
# print(X_Validation_new.shape)
# Y_Validation = np.load(data_path+'vali_label.npy')[0:8,:,:,:]
# print(Y_Validation.shape)
# name += "40_samples_"
# # ##############################################

# # ##############################################
# # 30 samples 
# X_train_new = X_train[0:24,:,:,:]
# print(X_train_new.shape)
# Y_train = np.load(data_path+'train_label.npy')[0:24,:,:,:]
# print(Y_train.shape)

# X_Validation_new = X_Validation[0:6,:,:,:]
# print(X_Validation_new.shape)
# Y_Validation = np.load(data_path+'vali_label.npy')[0:6,:,:,:]
# print(Y_Validation.shape)
# name += "30_samples_"
# # ##############################################

# ##############################################
# # 20 samples 
# X_train_new = X_train[0:16,:,:,:]
# print(X_train_new.shape)
# Y_train = np.load(data_path+'train_label.npy')[0:16,:,:,:]
# print(Y_train.shape)

# X_Validation_new = X_Validation[0:4,:,:,:]
# print(X_Validation_new.shape)
# Y_Validation = np.load(data_path+'vali_label.npy')[0:4,:,:,:]
# print(Y_Validation.shape)
# name += "20_samples_"
# ##############################################


##############################################
# # 10 samples 
# X_train_new = X_train[0:8,:,:,:]
# print(X_train_new.shape)
# Y_train = np.load(data_path+'train_label.npy')[0:8,:,:,:]
# print(Y_train.shape)

# X_Validation_new = X_Validation[0:2,:,:,:]
# print(X_Validation_new.shape)
# Y_Validation = np.load(data_path+'vali_label.npy')[0:2,:,:,:]
# print(Y_Validation.shape)
# name += "10_samples_"
##############################################


# ##############################################
# # 1200 samples 
# X_train_new = X_train
# print(X_train_new.shape)
# Y_train = np.load(data_path+'train_label.npy')
# print(Y_train.shape)

# X_Validation_new = X_Validation
# print(X_Validation_new.shape)
# Y_Validation = np.load(data_path+'vali_label.npy')
# print(Y_Validation.shape)
# # ##############################################

(8, 224, 224, 8)
(8, 224, 224, 1)
(2, 224, 224, 8)
(2, 224, 224, 1)


## **With NAIP**

In [4]:
# The dataset has 8 channels:
# 0. Curvature
# 1. Slope
# 2. Openness
# 3. DEM
# 4. TPI 21
# 5. Reflectance (LiDAR intensity)
# 6. Geomorphon
# 7. TPI 9
# but the model expects 8 channels
# So we exclude TPI_9 channel from the data set

#Without NAIP
# data_path = 'Covington_data/without_NAIP/nodata_as_0/'

#With NAIP
# data_path = 'Covington_data/include_NAIP/nodata_as_0/100_samples/'
data_path = 'Covington_data/include_NAIP/nodata_as_0/350_samples/'
# data_path = 'Covington_data/include_NAIP/nodata_as_0/500_samples/'

# read in training and validation sample patches
X_train_new = np.load(data_path+'train_data.npy')
X_Validation_new = np.load(data_path+'vali_data.npy')
print(X_train_new.shape)
print(X_Validation_new.shape)

#Read training and validation labels
Y_Validation = np.load(data_path+'vali_label.npy')
Y_train = np.load(data_path+'train_label.npy')

#Cast both labales to float32
Y_Validation = Y_Validation.astype(np.float32)
Y_train = Y_train.astype(np.float32)


(524, 224, 224, 8)
(176, 224, 224, 8)


# **Note:**  

Training from scratch   
- Total params: 53,508,217  
- Trainable params: 53,490,169  
- Non-trainable params: 18,048  
        
**Without NAIP (dropr rate: 0.2)**  
The training takes 17 seconds for each epoch  
- First training: the training stop at Epoch 105 (history_train_from_scratch_NoNAIP_20201012-111316)
- We continue for 33 epochs more (history_train_from_scratch_NoNAIP_20201012-112416)
    - Training set accu: 99.92%
    - Validation set accu: 98.52%     

**last line of log**   
Epoch 33/400  
50/50 [==============================] - 16s 313ms/step - loss: -0.9859 - dice_coef: 0.9859 - accuracy: 0.9992 - val_loss: -0.5574 - val_dice_coef: 0.5574 - val_accuracy: 0.9852
    
**With NAIP (dropr rate: 0.2)**   
The training takes 17 seconds for each epoch (total 38 minutes)
- First training: the training stop at Epoch 141 (history_train_from_scratch_NAIP_20201012-121601)
    - Training set accu: 99.91%
    - Validation set accu: 98.87%   

**last line of log**     
Epoch 141/400   
50/50 [==============================] - 16s 321ms/step - loss: -0.9849 - dice_coef: 0.9849 - accuracy: 0.9991 - val_loss: -0.6443 - val_dice_coef: 0.6443 - val_accuracy: 0.9887   

In [19]:
import pickle
import time
timestr = time.strftime("%Y%m%d-%H%M%S")
root_path = ''
# save the trained model
model_yaml = model.to_yaml()
with open(root_path+"train_from_scratch_NAIP_"+timestr+".yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
# save the weights
model.save(root_path+"train_from_scratch_NAIP_"+timestr+".h5")
# save the intermdediate results and training statistics
with open(root_path+"history_train_from_scratch_NAIP_"+timestr+".pickle", 'wb') as file_pi:
    pickle.dump(no_transfer_learning_results.history, file_pi, protocol=2)

### Will wait for the whole area data to do prediction

In [13]:
# Save the predicted labels.
X_test = np.load('Covington_data/without_NAIP/prediction_data_07092020.npy')
preds_test = loaded_model.predict(X_test)
preds_test_t = (preds_test > 0.5).astype(np.uint8)
np.save('Covington_data/without_NAIP/pred_Rowan_mdl_Covington_data.npy',preds_test_t)

In [4]:
import time 

start = time.time()

import pandas as pd
import numpy as np
df = pd.DataFrame(np.random.randint(1,9999,size=(10000000, 1)), columns=['Random numbers'])
df['Random numbers'] = df['Random numbers'].astype(str)

print(f"Finished saving! It takes {(time.time() - start)/60} minutes")

Finished saving! It takes 0.1023483157157898 minutes
